In [1]:
!pip install langchain langchain-text-splitters langchain-community bs4

  Using cached langchain-1.0.3-py3-none-any.whl.metadata (4.7 kB)
  Using cached langchain_text_splitters-1.0.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
  Using cached langchain_core-1.0.3-py3-none-any.whl.metadata (3.5 kB)
  Using cached langgraph-1.0.2-py3-none-any.whl.metadata (7.4 kB)
  Using cached pydantic-2.12.3-py3-none-any.whl.metadata (87 kB)
  Using cached langchain_classic-1.0.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-win_amd64.whl.metadata (2.4 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.11.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached langsmith-0.4.40-py3-none-any.whl.metadata (14 kB)
  Using cached htt


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Setup

In [2]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
!pip install -U "langchain[openai]"

  Using cached langchain_openai-1.0.2-py3-none-any.whl.metadata (1.8 kB)
  Using cached openai-2.7.1-py3-none-any.whl.metadata (29 kB)
  Using cached tiktoken-0.12.0-cp312-cp312-win_amd64.whl.metadata (6.9 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.11.1-cp312-cp312-win_amd64.whl.metadata (5.3 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached regex-2025.11.3-cp312-cp312-win_amd64.whl.metadata (41 kB)
Using cached langchain_openai-1.0.2-py3-none-any.whl (81 kB)
Using cached openai-2.7.1-py3-none-any.whl (1.0 MB)
Using cached tiktoken-0.12.0-cp312-cp312-win_amd64.whl (878 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached jiter-0.11.1-cp312-cp312-win_amd64.whl (204 kB)
Using cached regex-2025.11.3-cp312-cp312-win_amd64.whl (277 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4.1")

In [5]:
!pip install -U "langchain-openai"


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [7]:
!pip install -U "langchain-core"


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

## Document Loading

In [10]:
!pip install pypdf

  Using cached pypdf-6.1.3-py3-none-any.whl.metadata (7.1 kB)
Using cached pypdf-6.1.3-py3-none-any.whl (323 kB)



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from langchain_community.document_loaders import PyPDFLoader

# Get the absolute path to the PDF file
pdf_path = os.path.join(os.path.dirname(os.getcwd()), "docs", "AEM1.pdf")
loader = PyPDFLoader(pdf_path)
pages = loader.load()

In [12]:
# number of pages loaded (the PDF has 43 pages)
len(pages)

43

In [13]:
# Let's see the third page (Editor's Letter)
page3 = pages[2]
print(page3.page_content[:500])  # print the first 500 characters

prosperoenglish.com www.youtube.com/c/ProsperoEnglish 
 
3 Your American English Magazine | 1/2022 
 
Editor’s Letter 
Hey there, I’m excited to deliver this very first 
issue of Your American English Magazine to you. I 
don’t know how this new magazine will be doing. 
Hopefully well, because if there is interest in it, I’ll 
be publishing more issues on a regular basis. 
So, what will you find in this issue? First, there’s a 
story, A Winter Hike. I’m planning to write a story 
like that in eac


In [14]:
# Let's see the metadata of the third page
page3.metadata

{'producer': 'Microsoft® Office Word 2007',
 'creator': 'Microsoft® Office Word 2007',
 'creationdate': '2022-02-17T14:01:22+01:00',
 'title': 'prosperoenglish.com',
 'author': 'Victor',
 'moddate': '2022-02-17T14:01:22+01:00',
 'source': 'd:\\Projects\\langchain_RAG_PDF\\docs\\AEM1.pdf',
 'total_pages': 43,
 'page': 2,
 'page_label': '3'}

## Document Splitting

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True
)

docs = text_splitter.split_documents(pages)

In [16]:
# number of documents after splitting
len(docs)

98

## Embedding and Storing

In [17]:
doc_ids = vector_store.add_documents(documents=docs)
print(len(doc_ids))

98


In [18]:
print(doc_ids[0])

499d3d77-3c4d-40e2-92b4-68f00bdd7a77


## RAG Agent

In [19]:
from langchain.tools import tool

In [20]:
# RAG agent
@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [21]:
from langchain.agents import create_agent

In [22]:
tools = [retrieve]

system_prompt = (
    "You have access to a tool that retrieves context from a PDF document. "
    "Use it to better answer user queries."
)

agent = create_agent(model, tools, system_prompt=system_prompt)

## Generation

In [23]:
query = (
    "Who are Frieda, Borg, Kev, Ike and Bree to one another?\n\n"
    "How old are they?\n\n"
    "What is the reason why Kev and Ike initially didn't go with their folks for a hike."
)

In [24]:
for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values"
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Who are Frieda, Borg, Kev, Ike and Bree to one another?

How old are they?

What is the reason why Kev and Ike initially didn't go with their folks for a hike.
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_qgrc02mCuY14NQoRN9XVUOWy)
 Call ID: call_qgrc02mCuY14NQoRN9XVUOWy
  Args:
    query: Who are Frieda, Borg, Kev, Ike and Bree to one another?
  retrieve (call_YBvPgsqdIsqKlTHSBwzKteDl)
 Call ID: call_YBvPgsqdIsqKlTHSBwzKteDl
  Args:
    query: How old are Frieda, Borg, Kev, Ike and Bree?
  retrieve (call_nkUW6W6iBxemTxfEyrH76Sxy)
 Call ID: call_nkUW6W6iBxemTxfEyrH76Sxy
  Args:
    query: Why did Kev and Ike initially not go with their folks for a hike?
================================= Tool Message =================================
Name: retrieve

Source: {'producer': 'Microsoft® Office Word 2007', 'creator': 'Microsoft® Office Word 2007', '